In [ ]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler

In [ ]:
def load_dataset(csv_path):
    df = pd.read_csv(csv_path, sep= ';')
    required_columns = ['Azimuth', 'Altitude', 'Timestamp']

    if not all (col in df.columns for col in required_columns):
        raise ValueError("Required columns are not present in the DataFrame.")
    
    X = df[['Timestamp','Azimuth','Altitude']].values
    y = df[['Azimuth','Altitude']].values

    return X, y, df

In [ ]:
csv_path = 'cleaned/cleaned_data_2014-2016.csv'
X, y, df = load_dataset(csv_path)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2 , random_state= 42)

In [ ]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train)
X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test)

In [ ]:
timesteps = 1
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, X_train_scaled.shape[1]))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], timesteps, X_test_scaled.shape[1]))

In [ ]:
model = Sequential()
model.add(LSTM(units=128, activation='relu', input_shape=(timesteps, X_train_scaled.shape[1]), return_sequences=True))
model.add(LSTM(units=64, activation='relu', return_sequences=True))
model.add(LSTM(units=32, activation='relu', return_sequences=True))
model.add(LSTM(units=8, activation='relu'))
model.add(Dense(units=2))

optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])  

history = model.fit(X_train_reshaped, y_train_scaled, epochs=100, batch_size=1024, validation_data=(X_test_reshaped, y_test_scaled))

In [ ]:
predicted_values_scaled = model.predict(X_test_reshaped)
predicted_values = scaler_y.inverse_transform(predicted_values_scaled)
mse = mean_squared_error(y_test_scaled, predicted_values_scaled)
r2 = r2_score(y_test_scaled, predicted_values_scaled)

accuracy = model.evaluate(X_test_reshaped, y_test_scaled)

In [ ]:
timestamps_test = X_test[:, 0]
df_results = pd.DataFrame({
    'Timestamp': timestamps_test,
    'Actual_Azimuth': y_test[:, 0],
    'Actual_Altitude': y_test[:, 1],
    'Predicted_Azimuth': predicted_values[:, 0],
    'Predicted_Altitude': predicted_values[:, 1]
})

In [ ]:
output_csv_path = 'azimuth_altitude_prediction_results_2years.csv'
df_results.to_csv(output_csv_path, index=False)

In [ ]:
print(f"Mean Squared Error on Test Data: {mse}")
print(f"R-squared on Test Data: {r2}")
print(f"Accuracy on Test Data: {accuracy}")
print(f"Results saved to: {output_csv_path}")